In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout

data_train, data_test, data_val=split_data2(data, 0.2, 0.2)

evaluator_validation = EvaluatorHoldout(data_val, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(data_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2213 (17.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2630 (20.8%) Users that have less than 1 test interactions


In [4]:
study_name = "LightFMCF"
study = optuna.create_study(
    study_name=study_name,
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-04 16:25:52,378] A new study created in memory with name: LightFMCF


In [5]:
#from Recommenders.FactorizationMachines.LightFMRecommender import LightFMCFRecommender
#from lightfm import LightFM
#
#recommender = LightFMCFRecommender(data_train)
#recommender.fit(epochs = 10)
#_, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)

In [5]:
from Recommenders.FactorizationMachines.LightFMRecommender import LightFMCFRecommender
from lightfm import LightFM

def objective(trial):
    #loss=trial.suggest_categorical("loss", ["bpr", "warp", "warp-kos"])
    #sgd_mode=trial.suggest_categorical("sgd_mode", ["adagrad", "adadelta"])
    #n_components=trial.suggest_int('n_components', 0, 100)
    #item_alpha=trial.suggest_float('item_alpha', 0, 1.0)
    #user_alpha=trial.suggest_float('user_alpha', 0, 1.0)
    #learning_rate=trial.suggest_float('learning_rate', 0, 1.0)
    #num_threads=1

    loss = "bpr"
    sgd_mode = "adagrad" 
    n_components = 10
    item_alpha = 0.0
    user_alpha = 0.0
    learning_rate = 0.05
    
    recommender = LightFMCFRecommender(data_train)
    recommender.fit(
        loss=loss,
        sgd_mode=sgd_mode,
        n_components=n_components,
        item_alpha=item_alpha,
        user_alpha=user_alpha,
        learning_rate=learning_rate)

    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=10)

: 

In [ ]:
study.best_params